Desarrollo de modelo ML en relación ítem-ítem, utilizando el algoritmo de k-NearestNeighbors. Tomando como elementos para medir la sercania de los juegos los generos de df_games, y si son recomendados o no utilizando la columna 'recomemend' y 'sentiment_analysis' de df_review.

In [2]:
import pandas as pd
import numpy as np

In [4]:
from sklearn.neighbors import NearestNeighbors

In [7]:

df_review = pd.read_csv('df_reviews.csv')
df_games = pd.read_csv('df_games.csv')

Para el modelo se van utilizar las colunas de generos, recomendaciones y analisis de sentimiento. Las cuales estan todas el variables numericas. Unimos los df_games y df_review y seleccionamos las columnas a utilizar

In [8]:
# uno los df dejando afura la columna de años de publicacion de df_review ya qeu no es necesaria
df_merged = pd.merge(df_games, df_review.drop(columns=['año_publicado'], axis=1), on='item_id', how='outer')



In [9]:
# Selecciono columnas relevantes para el entrenamiento
columnas_generos = df_games.columns.difference(['app_name', 'item_id', 'año']).tolist()
columnas_recomendado = ['recommend']  
columnas_sentimiento = ['sentiment_analysis'] 



In [10]:
# combino las columnas y elimino los nulos que se podrian haber formado por las dudas
matriz_caracteristicas_combinada = df_merged[columnas_generos + columnas_recomendado + columnas_sentimiento].dropna()

# entreno el modelo NearestNeighbors con la matriz combinada
nn_model = NearestNeighbors(metric='cosine')
nn_model.fit(matriz_caracteristicas_combinada)

NearestNeighbors(metric='cosine')

Finalmente incorporamos el modelo a la funcion de recomendacion

In [11]:


def obtener_recomendaciones(id_juego, df_games, df_review, n=5):
    # Fusionar los DataFrames en uno solo
    df_merged = pd.merge(df_games, df_review.drop(columns=['año_publicado'], axis=1), on='item_id', how='outer')

    # Seleccionar columnas relevantes para el entrenamiento
    columnas_generos = df_games.columns.difference(['app_name', 'item_id', 'año']).tolist()
    columnas_recomendado = ['recommend']  # Suponiendo 'recomendado' es la columna de interés de df_review
    columnas_sentimiento = ['sentiment_analysis']  # Suponiendo 'analisis_de_sentimiento' es la columna de interés de df_review

    # Obtener matriz de características combinada
    matriz_caracteristicas_combinada = df_merged[columnas_generos + columnas_recomendado + columnas_sentimiento].dropna()

    # Entrenar el modelo NearestNeighbors con la matriz combinada
    nn_model = NearestNeighbors(metric='cosine')
    nn_model.fit(matriz_caracteristicas_combinada)

    # Buscar los vecinos más cercanos del juego dado
    if id_juego not in df_games['item_id'].values:
        return "ID de juego no encontrado en el DataFrame"

    idx = df_games.index[df_games['item_id'] == id_juego][0]
    idx = matriz_caracteristicas_combinada.index.get_loc(idx)  # Índice en la matriz de características

    # Encontrar los vecinos más cercanos
    distances, indices = nn_model.kneighbors([matriz_caracteristicas_combinada.iloc[idx]], n_neighbors=n+1)

    # Excluir el juego en sí mismo de las recomendaciones
    indices = indices.flatten()[1:]

    # Obtener los juegos recomendados
    juegos_recomendados = df_games.iloc[indices][['item_id', 'app_name']]
    recomendaciones = juegos_recomendados.to_dict(orient='records')

    return recomendaciones


In [12]:
obtener_recomendaciones(352760, df_games, df_review)

c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


[{'item_id': 498550, 'app_name': 'Depths of Dread'},
 {'item_id': 436664, 'app_name': 'Rocksmith® 2014 – Flyleaf Song Pack'},
 {'item_id': 525580, 'app_name': 'Spectrum Soundtrack'},
 {'item_id': 57200, 'app_name': 'Puzzle Dimension'},
 {'item_id': 265790, 'app_name': 'Residue: Final Cut'}]